In [1]:
# import stuff
import pennylane as qml
import pandas as pd
import sklearn

from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder


# Cancer dataset has 569 observations, nearest power of 2 is 2^10, so set n_qubits = 10
n_qubits = 10


# define the device
dev = qml.device("default.qubit", wires=range(n_qubits))

In [2]:
path = './wdbc.data'


# Documentation for the col_names can be found in wdbc.names
col_names = ["ID", "diagnosis", "radius", "texture", "perimeter", "area", "smoothness", "compactness", "concavity", "concave_points", "symmetry", "fractal dimension",
             "SE_radius", "SE_texture", "SE_perimeter", "SE_area", "SE_smoothness", "SE_compactness", "SE_concavity", "SE_concave_points", "SE_symmetry", "SE_fractal dimension",
             "worst_radius", "worst_texture", "worst_perimeter", "worst_area", "worst_smoothness", "worst_compactness", "worst_concavity", "worst_concave_points", "worst_symmetry", "worst_fractal dimension"]


def preprocess_data(data_path, col_names):
    data = pd.read_csv(path, names=col_names)
    input_data = data[["radius", "texture", "perimeter", "area", "smoothness", "compactness", "concavity", "concave_points", "symmetry", "fractal dimension",
             "SE_radius", "SE_texture", "SE_perimeter", "SE_area", "SE_smoothness", "SE_compactness", "SE_concavity", "SE_concave_points", "SE_symmetry", "SE_fractal dimension",
             "worst_radius", "worst_texture", "worst_perimeter", "worst_area", "worst_smoothness", "worst_compactness", "worst_concavity", "worst_concave_points", "worst_symmetry", "worst_fractal dimension"]]
    input_data = input_data.to_numpy()
    
    output_data = data[["diagnosis"]]
    output_data = np.ravel(output_data.to_numpy())
    
    label_encoder = LabelEncoder()
    output_data_encoded = label_encoder.fit_transform(output_data)
    
    X = input_data
    Y = output_data_encoded
    
    return X, Y

X, Y = preprocess_data(path, col_names)

In [3]:
print(X[1,0:4])
print(X[2,0:4])
print(np.linalg.norm(X[1,0:4]))
print(np.linalg.norm(X[2,0:4]))
print(X[1,0:4]/np.linalg.norm(X[1,0:4]))
print(X[2,0:4]/np.linalg.norm(X[2,0:4]))
print([X[1,0:4], X[2,0:4]])

[  20.57   17.77  132.9  1326.  ]
[  19.69   21.25  130.   1203.  ]
1332.9205932087627
1210.350469326963
[0.01543228 0.01333163 0.09970586 0.99480795]
[0.01626802 0.0175569  0.10740691 0.99392699]
[array([  20.57,   17.77,  132.9 , 1326.  ]), array([  19.69,   21.25,  130.  , 1203.  ])]


In [ ]:
def reshape_weights(n_qubits):
    

In [4]:
# Now we call amplitude embedding routine from Pennylane

@qml.qnode(dev)
def encoding_circuit(weights, features):
    qml.AmplitudeEmbedding(features, pad_with=0, normalize=True, wires=range(n_qubits))

    
    return qml.state()

encoding_circuit(X)

tensor([[0.00792541+0.j, 0.00457286+0.j, 0.054099  +0.j, ...,
         0.        +0.j, 0.        +0.j, 0.        +0.j],
        [0.00866576+0.j, 0.00748617+0.j, 0.05598828+0.j, ...,
         0.        +0.j, 0.        +0.j, 0.        +0.j],
        [0.00936668+0.j, 0.01010879+0.j, 0.06184199+0.j, ...,
         0.        +0.j, 0.        +0.j, 0.        +0.j],
        ...,
        [0.01164388+0.j, 0.0196964 +0.j, 0.07596583+0.j, ...,
         0.        +0.j, 0.        +0.j, 0.        +0.j],
        [0.00923021+0.j, 0.01314184+0.j, 0.06277436+0.j, ...,
         0.        +0.j, 0.        +0.j, 0.        +0.j],
        [0.02310989+0.j, 0.07308204+0.j, 0.14270951+0.j, ...,
         0.        +0.j, 0.        +0.j, 0.        +0.j]], requires_grad=True)

In [ ]:
def variational_circuit(weights):
    U3_weights, control_weights = reshape_weights(weights)
    for idx, element in enumerate(U3_weights):
        qml.U3(element[0], element[1], element[2], idx)

    for idx in range(n_qubits):
        c_k = 